In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from linescanning import (
    prf,
    optimal,
    pycortex,
    plotting,
    fitting,
)
import numpy as np
import os
import pandas as pd
from scipy import io
import cortex as cx
import re
import nibabel as nib
import matplotlib as mpl

opd = os.path.dirname
opj = os.path.join

<h1>Perform ROI drawing based on polar angle map</h2>
This notebook may be used to perform ROI drawing of V1/V2/V3 in Freeview. Having the data as a .label file makes integrating it in prfpy easier. After doing this in Freeview, you can load the data as a mask in Inkscape. When you draw around it, you can create a mask that is easy to use in pycortex as well for visualization.

In [19]:
sub=
ses='ses-avg'
model='gauss'
deriv='/data1/projects/Meman1/projects/pilot/derivatives'
filenamepattern = r'sub-\d+_ses-\d+_task-2R_run-\d+_(?:hemi-(L|R)_space-fsnative|space-fsnative_hemi-(L|R))_desc-denoised_bold\.npy'
fs_dir=opj(deriv, 'freesurfer')

minecc=0 #The minimum
maxecc=5 #The maximum eccentricity you want plotted in Freeview or Inkscape

<h3>1. Load previously fitted prf model</h3>


In [ ]:
pkl_file = opj(deriv, 'prf', sub, f'{sub}_{ses}_task-2R_model-{model}_stage-iter_desc-prf_params.pkl')      # Location and file name of the pkl file containing fitting parameters. Sometimes there is another 'ses'-folder in the subject folder, but not with 'avg' session
prf_params = prf.Parameters(pkl_file, model=model).to_df()                                                  # Load the parameters as a dataframe. NOTE: Header contains info about what the values mean
prf_obj_g = optimal.pRFCalc(pkl_file, model=model)

# Define for drawing:
prf_ecc = prf_params['ecc'].to_numpy()
prf_angle = prf_params['polar'].to_numpy()
prf_r2 = prf_params['r2'].to_numpy()
prf_ampl = prf_params['prf_ampl'].to_numpy()

# Let's look at the parameters for a bit:
display(prf_params.head())

print('Maximum eccentricity present in the file is:', prf_params['ecc'].max())
print('Minimum R2 present in the data is:', prf_params['r2'].min())

Now, create mean time series array:

In [ ]:
# Get mean signal intensity array
counter=0

for i, session in enumerate(["ses-2", "ses-3"]):                      #loop over functional sessions
            ses_folder_pybest  = os.path.join(deriv, 'pybest', sub, session, 'unzscored')

            if os.path.exists(ses_folder_pybest):

                sessionfile = sorted([filename for filename in sorted(os.listdir(ses_folder_pybest)) if re.match(filenamepattern, filename)])

                for run_nr in range(0, len(sessionfile), 2): #loop over the individual runs. IMPORTANT: REQUIRES APPROPRIATE FILE NUMBERING WITHIN FOLDER SO THAT ALPHABETICAL ORDER CAN BE USED FOR INDEXING
                    hemi_L = np.load(os.path.join(ses_folder_pybest,sessionfile[run_nr]))
                    hemi_R = np.load(os.path.join(ses_folder_pybest,sessionfile[run_nr+1]))
                    tseries = np.concatenate([hemi_L.T, hemi_R.T]) #concatenate the data
                    
                    try:
                        summedtseries += tseries.mean(axis=1)
                        counter += 1
                        
                    except NameError:
                        summedtseries = np.zeros(len(tseries.mean(axis=1))) 
                        summedtseries += tseries.mean(axis=1)
                        counter += 1

meantseries = summedtseries/counter

Now, we should create a mask for the visualization. You can do this as you wish. 

In [ ]:
# ** Control visibility of data [using mask] 
# If you don't want to show the values of every point (for example because it is outside the visual cortex). You may want to hide it. 
# If you are plotting PRFs, then you may want to hide the bad fits. So you can create a mask for where the rsq<threshold (e.g., 0.1)
# data_mask: what to show (TRUE), what to hide (FALSE)
# -> should boolean 1D np.ndarray, where the length = number of vertices in subject surface
# -> if unspecified, all surface functions assume TRUE for all voxels

prf_r2_mask = prf_params['r2'].to_numpy()>0.1  # Only use vertices with a R2 > 10%
prf_final_mask = prf_r2_mask

print(f'Final mask for {sub} {ses} {model} model contains {np.sum(prf_final_mask)} vertices out of a total of {len(prf_final_mask)} vertices')

<h3>2. Create Freesurfer custom overlay file for polar angle and eccentricity and mean signal intensity</h3>

In [ ]:
from dag_prf_utils.fs_tools import FSMaker
fs = FSMaker(sub=sub,fs_dir=fs_dir)

In [ ]:
# Add polar angle plot
fs.add_surface(
    data = prf_angle,
    surf_name = f'{sub}-polar_angle',    
    vmin = -3.14, vmax=3.14, # min and max values of polar anlge 
    data_mask=prf_final_mask,
    cmap = 'marco_pol', # A colleague (Marco Aqil) suggested this custom color map for polar angles. I called it 'marco_pol'
)

# Add eccentricity
fs.add_surface(
    data = prf_ecc,
    surf_name = f'{sub}-eccentricity',    
    vmin = minecc, vmax = maxecc, # min and max values of eccentricity
    data_mask=prf_final_mask,
    cmap = 'nipy_spectral', 
)

# Add r2
fs.add_surface(
    data = prf_r2,
    surf_name = f'{sub}-r2',    
    vmin = 0.1, vmax = 1, # min and max values of r2
    data_mask=prf_final_mask,
    cmap = 'plasma', 
)

# Add amplitude
fs.add_surface(
    data = prf_ampl,
    surf_name = f'{sub}-prfampl',    
    vmin = 0, # min value
    data_mask=prf_final_mask,
    cmap = 'viridis', 
)

# Add mean signal intensity
fs.add_surface(
    data = meantseries,
    surf_name = f'{sub}-signalintensity',
    cmap = 'seismic',
)

<h3>2b. Get MMP atlas in subject space</h3>

In [ ]:
!mri_surf2surf --srcsubject fsaverage --trgsubject {sub} --hemi lh --sval-annot {fs_dir}/fsaverage/label/lh.PALS_B12_Visuotopic.annot --tval {fs_dir}/{sub}/label/lh.PALS_B12_Visuotopic.annot
!mri_surf2surf --srcsubject fsaverage --trgsubject {sub} --hemi rh --sval-annot {fs_dir}/fsaverage/label/rh.PALS_B12_Visuotopic.annot --tval {fs_dir}/{sub}/label/rh.PALS_B12_Visuotopic.annot 


<h3>3. Draw ROIs in Freeview</h3>
N.B. run this locally, as Minerva on Spinoza computers can't open freeview. You can use the signal intensity overlay as a guide to see in which parts the polar angle data may not be trusted.

In [20]:
!freeview -f  {fs_dir}/{sub}/surf/lh.inflated:overlay={fs_dir}/{sub}/surf/custom/lh.{sub}-eccentricity::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-eccentricity_overlay::overlay={fs_dir}/{sub}/surf/custom/lh.{sub}-polar_angle::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-polar_angle_overlay::overlay={fs_dir}/{sub}/surf/custom/lh.{sub}-signalintensity::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-signalintensity_overlay::overlay={fs_dir}/{sub}/surf/custom/lh.{sub}-r2::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-r2_overlay::overlay={fs_dir}/{sub}/surf/custom/lh.{sub}-prfampl::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-prfampl_overlay::annot={fs_dir}/{sub}/label/lh.PALS_B12_Visuotopic.annot\
{fs_dir}/{sub}/surf/rh.inflated:overlay={fs_dir}/{sub}/surf/custom/rh.{sub}-eccentricity::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-eccentricity_overlay::overlay={fs_dir}/{sub}/surf/custom/rh.{sub}-polar_angle::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-polar_angle_overlay::overlay={fs_dir}/{sub}/surf/custom/rh.{sub}-signalintensity::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-signalintensity_overlay::overlay={fs_dir}/{sub}/surf/custom/rh.{sub}-r2::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-r2_overlay::overlay={fs_dir}/{sub}/surf/custom/rh.{sub}-prfampl::overlay_custom={fs_dir}/{sub}/surf/custom/{sub}-prfampl_overlay::annot={fs_dir}/{sub}/label/rh.PALS_B12_Visuotopic.annot\
--camera Azimuth 90 Zoom 1 Elevation 0 Roll 0 --colorscale  --verbose


RAS: 47.6131 -59.6380 57.0526
SurfaceRAS: 45.7310 -78.5271 18.0346
SurfaceRAS: 45.7310 -78.5271 18.0346
RAS: 41.1818 -61.2665 72.9849
SurfaceRAS: 39.2997 -80.1555 33.9669
SurfaceRAS: 39.2997 -80.1555 33.9669
RAS: 33.7364 -58.6008 55.0671
SurfaceRAS: 31.8543 -77.4899 16.0491
SurfaceRAS: 31.8543 -77.4899 16.0491
RAS: 44.1959 -61.1749 46.4714
SurfaceRAS: 42.3138 -80.0639 7.4534
SurfaceRAS: 42.3138 -80.0639 7.4534

RAS: 34.5014 -58.2379 50.9742
SurfaceRAS: 32.6193 -77.1269 11.9562
SurfaceRAS: 32.6193 -77.1269 11.9562
RAS: 44.6966 -64.6958 81.6452
SurfaceRAS: 42.8145 -83.5849 42.6272
SurfaceRAS: 42.8145 -83.5849 42.6272
RAS: 38.3456 -61.1337 51.1964
SurfaceRAS: 36.4635 -80.0228 12.1784
SurfaceRAS: 36.4635 -80.0228 12.1784
RAS: -38.3428 -60.3896 55.3178
SurfaceRAS: -40.2249 -79.2787 16.2998
SurfaceRAS: -40.2249 -79.2787 16.2998


<h3>4. Load ROI into Inkscape and draw pycortex ROI</h3>
Now, you need to go to Inkscape and load the mask of your ROI as data. Then, you can draw the ROI yourself. Annoyingly, you have to do this on Minerva because locally Inkscape does not work.

In [ ]:
#Load V1 vertices from left hemisphere, together with the number of vertices in the left hemisphere so indexing for right hemisphere works appropriately; the .label files start from 0 and work per hemisphere, whereas Inkscape merges the two hemispheres together.
V1_lh = nib.freesurfer.read_label(f"{deriv}/freesurfer/{sub}/customlabel/roidrawing/lh.V1.label")
V2_lh = nib.freesurfer.read_label(f"{deriv}/freesurfer/{sub}/customlabel/roidrawing/lh.V2.label")
V3_lh = nib.freesurfer.read_label(f"{deriv}/freesurfer/{sub}/customlabel/roidrawing/lh.V3.label")

all_lh = nib.freesurfer.read_geometry(f"{deriv}/freesurfer/{sub}/surf/lh.inflated")

#Load V1 vertices from right hemisphere. 
V1_rh = nib.freesurfer.read_label(f"{deriv}/freesurfer/{sub}/customlabel/roidrawing/rh.V1.label")
V2_rh = nib.freesurfer.read_label(f"{deriv}/freesurfer/{sub}/customlabel/roidrawing/rh.V2.label")
V3_rh = nib.freesurfer.read_label(f"{deriv}/freesurfer/{sub}/customlabel/roidrawing/rh.V3.label")

all_rh = nib.freesurfer.read_geometry(f"{deriv}/freesurfer/{sub}/surf/rh.inflated")

# Get the right indices for the right hemisphere by adding the number of vertices in left hemisphere to the ROI indices for the right hemisphere.
V1_rh = V1_rh + len(all_lh[0])
V2_rh = V2_rh + len(all_lh[0])
V3_rh = V3_rh + len(all_lh[0])

#Concatenate the 2 and sort them such that they are in the right order
V1_vertices = np.sort(np.concatenate([V1_lh, V1_rh]))
V2_vertices = np.sort(np.concatenate([V2_lh, V2_rh]))
V3_vertices = np.sort(np.concatenate([V3_lh, V3_rh]))

In Inkscape/Pycortex, the indexing of the mask works a little bit differently than in Freeview. So, fix it:

In [ ]:
# Create a ROI with the correct dimensions to create a Vertex from:
totalvertices = len(all_lh[0])+len(all_rh[0]) 

v1roi = np.zeros(totalvertices)
v2roi = np.zeros(totalvertices)
v3roi = np.zeros(totalvertices)

for vertex in V1_vertices:
    v1roi[vertex]=1

for vertex in V2_vertices:
    v2roi[vertex]=1

for vertex in V3_vertices:
    v3roi[vertex]=1

v1vertex = cx.Vertex(v1roi, subject=sub)
v2vertex = cx.Vertex(v2roi, subject=sub)
v3vertex = cx.Vertex(v3roi, subject=sub)

Now, load the mask into Inkscape. You need to trace the outline of the masks for all the ROIs. I have also added the eccentricity and polar angle and signal intensity such that you can see if everything makes sense on a flatmap.


In [ ]:
# In case you want to change eccentricty 
prf_obj_g.ecc_v.vmin1 = minecc
prf_obj_g.ecc_v.vmax1 = maxecc

# Add eccentricity and polar angle and signal intensity data. Remove these in the rois->shapes layers.
cx.utils.add_roi(prf_obj_g.ecc_v.get_result(), name='ecc', open_inkscape=False)
cx.utils.add_roi(prf_obj_g.polar_v.get_result(), name='polar', open_inkscape=False)
cx.utils.add_roi(cx.Vertex(meantseries, subject=sub, cmap='BuWtRd'), name='signalintensity', open_inkscape=False)

# Add V1/V2/V3 masks
cx.utils.add_roi(v1vertex, name='V1', open_inkscape=False)
cx.utils.add_roi(v2vertex, name='V2', open_inkscape=False)
cx.utils.add_roi(v3vertex, name='V3', open_inkscape=False)

<h3>5. Check results</h3>
Now, you should be done! For checking purposes, you can perform the following steps:
1. Make a pycortex webshow to see if the ROIs show up in the right way.
2. Check if the vertices included for V2 and V3 in Inkscape are the same/similar as in Freeview. For V1, this is not really possible because of the cut in the middle of V1 which makes you lose some vertices.

In [ ]:
#1. Check the pycortex webshow. Make sure to recache, otherwise ROIs will not show up.
cx.webshow({'polar': prf_obj_g.polar_v.get_result(), 'ecc': prf_obj_g.ecc_v.get_result(), 'r2': prf_obj_g.r2_v.get_result()}, recache=True)

In [ ]:
# 2. Check vertices included in the Freeview vs. Inkscape ROIs for V2 and V3:

# get vertices for ROIs as drawn with Inkscape
inkscape_roi_verts = cx.get_roi_verts(sub)

# roi_verts is a dictionary
inkscape_V2_verts = inkscape_roi_verts['V2']
inkscape_V3_verts = inkscape_roi_verts['V3']

print("V2...")
print("Freeview ROI has", len(V2_vertices), "vertices" )
print("Inkscape ROI has", len(inkscape_V2_verts), "vertices" )
print("Vertices only in Freeview ROI:", np.setdiff1d(V2_vertices, inkscape_V2_verts))
print("Vertices only in Inkscape ROI:", np.setdiff1d(inkscape_V2_verts, V2_vertices))
print("Total number of different vertices:", len(np.setdiff1d(V2_vertices, inkscape_V2_verts))+len(np.setdiff1d(inkscape_V2_verts, V2_vertices)))
print("")

print("V3...")
print("Freeview ROI has", len(V3_vertices), "vertices" )
print("Inkscape ROI has", len(inkscape_V3_verts), "vertices" )
print("Vertices only in Freeview ROI:", np.setdiff1d(V3_vertices, inkscape_V3_verts))
print("Vertices only in Inkscape ROI:", np.setdiff1d(inkscape_V3_verts, V3_vertices))
print("Total number of different vertices:", len(np.setdiff1d(V3_vertices, inkscape_V3_verts))+len(np.setdiff1d(inkscape_V3_verts, V3_vertices)))
